In [3]:
from bs4 import BeautifulSoup
import requests


In [14]:
url = "http://haikuguy.com/issa/search.php"
r = requests.get(url)
r.status_code

200

In [18]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(r.text, 'html.parser')

In [21]:
english = soup.find_all("p", {"class": "english"})

In [29]:
english_haikus = []
for p in english:
    text = p.text.replace("\n", " ").strip()
    english_haikus.append(text)

In [31]:
len(english_haikus)

10760

In [48]:
import random
import string

punct = string.punctuation
punct = punct.replace("'", "")
punct = punct.replace("-", "")

haikus_no_punct = []
for h in english_haikus:
    for p in punct:
        h = h.replace(p, "")
    haikus_no_punct.append(h)

In [142]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import models, layers, optimizers, losses, activations
from keras.utils import to_categorical
import keras
import numpy as np

In [103]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()

In [104]:
tokenizer.fit_on_texts(haikus_no_punct)

In [105]:
tokens = tokenizer.texts_to_sequences(haikus_no_punct) # convert to dense-encoded sequences
tokens = pad_sequences(tokens) # pad them to all have the same length

In [106]:
tokens.shape # verify that n_rows matches the number of haikus

(10760, 15)

In [ ]:
# train_split_end = 0.8 * tokens.shape[0]
# train_data, test_data = tokens[]

In [ ]:
vocab_size = len(tokenizer.index_word) + 1
embedding_dimension = 512
M = 256

In [144]:
X = tokens[:, :-1]
y = tokens[:,-1]
print(X.shape, y.shape)

(10760, 14) (10760,)


In [146]:
y_categorical = to_categorical(y, num_classes=vocab_size)
print(y_categorical.shape)

(10760, 7027)


In [152]:
i = layers.Input(shape=(X.shape[-1],))
x = layers.Embedding(vocab_size, embedding_dimension)(i)
x = layers.LSTM(M, return_sequences=True)(x)
x = layers.LSTM(M)(x)
x = layers.Dense(M, activation='relu')(x)
x = layers.Dropout(0.3)(x)
x = layers.BatchNormalization()(x)
o = layers.Dense(vocab_size, activation='softmax')(x)
model = models.Model(i, o)

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [153]:
model.fit(X, y_categorical, validation_split=0.2, epochs=10)

Epoch 1/10
269/269 [==============================] - 11s 32ms/step - loss: 8.4658 - val_loss: 7.1614
Epoch 2/10
269/269 [==============================] - 8s 29ms/step - loss: 6.3887 - val_loss: 7.0823
Epoch 3/10
269/269 [==============================] - 8s 29ms/step - loss: 6.0736 - val_loss: 7.1358
Epoch 4/10
269/269 [==============================] - 8s 29ms/step - loss: 5.8639 - val_loss: 7.1661
Epoch 5/10
269/269 [==============================] - 8s 29ms/step - loss: 5.6268 - val_loss: 7.2832
Epoch 6/10
269/269 [==============================] - 8s 29ms/step - loss: 5.4323 - val_loss: 7.4067
Epoch 7/10
 59/269 [=====>........................] - ETA: 5s - loss: 5.1934

KeyboardInterrupt: 

In [177]:
test_input = "Cold morning"
max_length = 14

while len(test_input.split()) < max_length:
    encoded_input = tokenizer.texts_to_sequences([test_input])
    encoded_input = pad_sequences(encoded_input, maxlen=14)
    model_output = model.predict(encoded_input)
    next_predicted_word = tokenizer.index_word[model_output.argmax()]
    test_input += f" {next_predicted_word}"

# encoded_text = tokenizer.texts_to_sequences([input_text])[0]
# pad_encoded = pad_sequences([encoded_text], maxlen=8, truncating='pre')
# print(encoded_text, pad_encoded)
# print((model.predict(pad_encoded)[0]).argsort()[::-1])
# for i in (model.predict(pad_encoded)[0]).argsort()[::-1]:
#   if i == 0:
#     break
#   pred_word = tokenizer.index_word[i]
#   print("Next word suggestion:",pred_word)

In [178]:
test_input

'Cold morning sun bloomed bloomed another now now gate gate gate gate gate green'

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2b46da0f-9118-4fef-a62c-5402bbd6e1e0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>